# Configurações

In [ ]:
%pip install -r ./../requirements.txt

# Import's

In [ ]:
from AlgoritmoGenetico import AlgoritmoGenetico, AlgoritmoGeneticoParalelo
import matplotlib.pyplot as plt
import statistics
import json

# Valores de Configuração

In [ ]:
qtd_execucoes = 10 ## Quantidade de execuções a serem realizadas por parametro, para realização da média de tempo gasto
random_seed = 7 ## Random seed a ser usada
lista_qtd_geracoes = [25, 50, 75, 100] ## Lista com a quantidade de gerações a serem realizadas
lista_qtd_individuos = [100, 250, 500, 1000] ## Lista com a quantidade de individuos a serem criados por geração
lista_qtd_processos = [2, 4, 6, 8] ## Lista com a quantidade de processos a serem utilizadas

# Criando as Instâncias dos Algoritmos Genéticos

In [ ]:
AG = AlgoritmoGenetico(random_seed=random_seed) ## Cria uma instância do Algoritmo Genético Sequencial
AGP = AlgoritmoGeneticoParalelo(random_seed=random_seed) ## Cria uma instância do Algoritmo Genético Paralelo

# Executando

In [ ]:
resultado_final = [] ## Armazena os valores resultantes das execuções

for geracoes in lista_qtd_geracoes: ## Percorre a lista pegando a quantidade de gerações

    print("- Inicializada a etapa com " + str(geracoes) + " gerações!")

    ## Configuração da quantidade de gerações dos Algoritmos Geneticos - Inicio
    AG.set_qtd_geracoes(geracoes) ## Set da quantidade de geração do AG Sequencial
    AGP.set_qtd_geracoes(geracoes) ## Set da quantidade de geração do AG Paralelo
    ############################# - Fim

    lista_resultados_individuos = []

    for individuos in lista_qtd_individuos: ## Percorre a lista pegando a quantidade de individuos

        print("\t- Inicializada a etapa com " + str(individuos) + " individuos!")

        ## Configuração da quantidade de individuos dos Algoritmos Geneticos - Inicio
        AG.set_qtd_individuos(individuos) ## Set da quantidade de individuos do AG Sequencial
        AGP.set_qtd_individuos(individuos) ## Set da quantidade de individuos do AG Paralelo
        ############################# - Fim

        print("\t\t- Inicializada a etapa de execução do Algoritmo Genético Sequencial!")

        ## Execução do Algoritmo Genético Sequencial - Inicio
        lista_resultados_sequenciais = [] ## Armazena o resultados das execuções do AG Sequencial com determinada configuração
        for execucao in range(qtd_execucoes): ## Executando o código X vezes
            lista_resultados_sequenciais.append(AG.start[1]) ## Salvando o tempo gasto na execução
        ############################# - Fim

        print("\t\t- Finalizada a etapa de execução do Algoritmo Genético Sequencial!")

        print("\t\t- Inicializada a etapa de execução do Algoritmo Genético Paralelo!")

        ## Execução do Algoritmo Genético Paralelo - Inicio
        lista_resultados_paralelos = [] ## Armazena o resultados das execuções do AG Paralelo com determinada configuração
        for processos in lista_qtd_processos: ## Percorre a lista pegando a quantidade de processos a serem utilizados
            lista_resultados_paralelos_processos = [] ## Armazena os resultados das execuções do AG Paralelo por processos
            
            ## Configuração da quantidade de processos dos Algoritmos Geneticos Paralelo - Inicio
            AGP.set_qtd_processos(processos) ## Set da quantidade de processos do AG Paralelo
            ############################# - Fim

            print("\t\t\t- Inicializada a execução com " + str(processos) + " processos!")

            for execucao in range(qtd_execucoes): ## Executando o código X vezes
                lista_resultados_paralelos_processos.append(AGP.start[1]) ## Salvando o tempo gasto na execução
            lista_resultados_paralelos.append(dict({"processos" : processos, "resultado" : statistics.mean(lista_resultados_paralelos_processos)}))
        
            print("\t\t\t- Finalizada a execução com " + str(processos) + " processos!")
        
        ############################# - Fim

        print("\t\t- Finalizada a etapa de execução do Algoritmo Genético Paralelo!")

        ## Preparando os resultados obtidos por individuo - Inicio
        lista_resultados_individuos.append(
            dict({
                "individuos" : individuos,
                "resultado_sequencial" : statistics.mean(lista_resultados_sequenciais),
                "resultado_paralelo" : lista_resultados_paralelos
            })
        )
        ############################# - Fim

        print("\t- Finalizada a etapa com " + str(individuos) + " individuos!")

    ## Preparando os resultados obtidos por individuo - Inicio
    resultado_final.append(
        dict({
            "geracao" : geracoes,
            "resultado" : lista_resultados_individuos
        })
    )
    ############################# - Fim

    print("- Finalizada a etapa com " + str(geracoes) + " gerações!")


# Salvando os Resultados

In [ ]:
with open("resultados.json", "w") as arquivo:
    json.dump(resultado_final, arquivo, indent=4)
    arquivo.close()

# Lendo os Resultados Salvos

In [ ]:
with open("resultados.json", "r") as arquivo:
    resultado_final = json.load(arquivo)
    arquivo.close()

# Observando os Resultados

In [ ]:
resultado_final

# Analisando os Resultados

## Gráficos de Tempo de execução

In [ ]:
for geracoes in resultado_final: ## Percorre as gerações utilizadas no resultado

    dados_grafico_echo_x = [[] for x in range(len(lista_qtd_processos)+1)] ## Armazena os dado o tempo gasto nas exceuções para plot do grafico
    dados_grafico_echo_y = [resultado["individuos"] for resultado in geracoes["resultado"]] ## Armazena os dado de individuos para plot no grafico

    for resultado in geracoes["resultado"]: ## Percorre os resultados

        index = 0
        dados_grafico_echo_x[index].append([resultado["resultado_sequencial"], "Sequencial"])

        for resultado_paralelo in resultado["resultado_paralelo"]:
            index += 1
            dados_grafico_echo_x[index].append([resultado_paralelo["resultado"], str( str(resultado_paralelo["processos"]) + "-Processos")])

    for dados_echo_x in dados_grafico_echo_x:
        echo_x = [dado[0] for dado in dados_echo_x]
        plt.plot(dados_grafico_echo_y, echo_x, label=dados_echo_x[0][1])
    plt.xlabel("Quantidade de individuos")
    plt.ylabel("Tempo de execução (segundos)")
    plt.title("Gráfico com " + str(geracoes["geracao"]) + " gerações")
    plt.grid(True)
    plt.legend()
    plt.show()